In [1]:
import onnxruntime as ort
import numpy as np
import pandas as pd
import collections
import functools

session = ort.InferenceSession("models\prosit\hla_cid\weight_192_0.16253_compatible.onnx")
input_names = [input.name for input in session.get_inputs()]
input_names

['peptide_sequences', 'precursor_charges', 'normalized_collision_energies']

In [2]:
# Constants from https://github.com/kusterlab/prosit/blob/master/prosit/constants.py
DATA_PATH = "/root/data.hdf5"
MODEL_SPECTRA = "/root/model_spectra/"
MODEL_IRT = "/root/model_irt/"
OUT_DIR = "/root/prediction/"

VAL_SPLIT = 0.8

TRAIN_EPOCHS = 500
TRAIN_BATCH_SIZE = 1024
PRED_BATCH_SIZE = 1024
PRED_BAYES = False
PRED_N = 100

TOLERANCE_FTMS = 25
TOLERANCE_ITMS = 0.35
TOLERANCE_TRIPLETOF = 0.5

TOLERANCE = {"FTMS": (25, "ppm"), "ITMS": (0.35, "da"), "TripleTOF": (50, "ppm")}


ALPHABET = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
    "M(ox)": 21,
}
ALPHABET_S = {integer: char for char, integer in ALPHABET.items()}

CHARGES = [1, 2, 3, 4, 5, 6]
DEFAULT_MAX_CHARGE = len(CHARGES)
MAX_FRAG_CHARGE = 3
MAX_SEQUENCE = 30
MAX_ION = MAX_SEQUENCE - 1
ION_TYPES = ["y", "b"]
NLOSSES = ["", "H2O", "NH3"]

FORWARD = {"a", "b", "c"}
BACKWARD = {"x", "y", "z"}

# Amino acids
MODIFICATION = {
    "CAM": 57.0214637236,  # Carbamidomethylation (CAM)
    "OX": 15.99491,  # Oxidation
}
AMINO_ACID = {
    "G": 57.021464,
    "R": 156.101111,
    "V": 99.068414,
    "P": 97.052764,
    "S": 87.032028,
    "U": 150.95363,
    "L": 113.084064,
    "M": 131.040485,
    "Q": 128.058578,
    "N": 114.042927,
    "Y": 163.063329,
    "E": 129.042593,
    "C": 103.009185 + MODIFICATION["CAM"],
    "F": 147.068414,
    "I": 113.084064,
    "A": 71.037114,
    "T": 101.047679,
    "W": 186.079313,
    "H": 137.058912,
    "D": 115.026943,
    "K": 128.094963,
}
AMINO_ACID["M(ox)"] = AMINO_ACID["M"] + MODIFICATION["OX"]

# Atomic elements
PROTON = 1.007276467
ELECTRON = 0.00054858
H = 1.007825035
C = 12.0
O = 15.99491463
N = 14.003074

# Tiny molecules
N_TERMINUS = H
C_TERMINUS = O + H
CO = C + O
CHO = C + H + O
NH2 = N + H * 2
H2O = H * 2 + O
NH3 = N + H * 3

NEUTRAL_LOSS = {"NH3": NH3, "H2O": H2O}

ION_OFFSET = {
    "a": N_TERMINUS - CHO,
    "b": N_TERMINUS - H,
    "c": N_TERMINUS + NH2,
    "x": C_TERMINUS + CO - H,
    "y": C_TERMINUS + H,
    "z": C_TERMINUS - NH2,
}

In [3]:
# Annotations from https://github.com/kusterlab/prosit/blob/master/prosit/annotate.py

def adjust_masses(method):
    if method == "SILAC":
        offsets = {"K": 8.01419881319, "R": 10.008268599}
    else:
        raise ValueError("Don't know method: " + method)

    for aa, offset in offsets.items():
        AMINO_ACID[aa] += offset


def get_mz(sum_, ion_offset, charge):
    return (sum_ + ion_offset + charge * PROTON) / charge


def get_mzs(cumsum, ion_type, z):
    return [get_mz(s, ION_OFFSET[ion_type], z) for s in cumsum[:-1]]


def get_annotation(forward, backward, charge, ion_types):
    tmp = "{}{}"
    tmp_nl = "{}{}-{}"
    all_ = {}
    for ion_type in ion_types:
        if ion_type in FORWARD:
            cummass = forward
        elif ion_type in BACKWARD:
            cummass = backward
        else:
            raise ValueError("unkown ion_type: {}".format(ion_type))
        masses = get_mzs(cummass, ion_type, charge)
        d = {tmp.format(ion_type, i + 1): m for i, m in enumerate(masses)}
        all_.update(d)
        for nl, offset in NEUTRAL_LOSS.items():
            nl_masses = get_mzs(cummass - offset, ion_type, charge)
            d = {tmp_nl.format(ion_type, i + 1, nl): m for i, m in enumerate(nl_masses)}
            all_.update(d)
    return collections.OrderedDict(sorted(all_.items(), key=lambda t: t[0]))

In [4]:
# Utils from from https://github.com/kusterlab/prosit/blob/master/prosit/utils.py

def check_mandatory_keys(dictionary, keys):
    for key in keys:
        if key not in dictionary.keys():
            raise KeyError("key {} is missing".format(key))
    return True


def reshape_dims(array, nlosses=1, z=3):
    return array.reshape([array.shape[0], MAX_ION, len(ION_TYPES), nlosses, z])


def get_sequence(sequence):
    d = ALPHABET_S
    return "".join([d[i] if i in d else "" for i in sequence])


def sequence_integer_to_str(array):
    sequences = [get_sequence(array[i]) for i in range(array.shape[0])]
    return sequences


def peptide_parser(p):
    p = p.replace("_", "")
    if p[0] == "(":
        raise ValueError("sequence starts with '('")
    n = len(p)
    i = 0
    while i < n:
        if i < n - 3 and p[i + 1] == "(":
            j = p[i + 2 :].index(")")
            offset = i + j + 3
            yield p[i:offset]
            i = offset
        else:
            yield p[i]
            i += 1

In [5]:
# Sanity utils from https://github.com/kusterlab/prosit/blob/master/prosit/sanitize.py

def reshape_dims(array):
    n, dims = array.shape
    assert dims == 174
    nlosses = 1
    return array.reshape(
        [array.shape[0], MAX_SEQUENCE - 1, len(ION_TYPES), nlosses, MAX_FRAG_CHARGE]
    )


def reshape_flat(array):
    s = array.shape
    flat_dim = [s[0], functools.reduce(lambda x, y: x * y, s[1:], 1)]
    return array.reshape(flat_dim)


def normalize_base_peak(array):
    # flat
    maxima = array.max(axis=1)
    array = array / maxima[:, numpy.newaxis]
    return array


def mask_outofrange(array, lengths, mask=-1.0):
    # dim
    for i in range(array.shape[0]):
        array[i, lengths[i] - 1 :, :, :, :] = mask
    return array


def cap(array, nlosses=1, z=3):
    return array[:, :, :, :nlosses, :z]


def mask_outofcharge(array, charges, mask=-1.0):
    # dim
    for i in range(array.shape[0]):
        if charges[i] < 3:
            array[i, :, :, :, charges[i] :] = mask
    return array


def get_spectral_angle(true, pred, batch_size=600):
    import tensorflow

    n = true.shape[0]
    sa = numpy.zeros([n])

    def iterate():
        if n > batch_size:
            for i in range(n // batch_size):
                true_sample = true[i * batch_size : (i + 1) * batch_size]
                pred_sample = pred[i * batch_size : (i + 1) * batch_size]
                yield i, true_sample, pred_sample
            i = n // batch_size
            yield i, true[(i) * batch_size :], pred[(i) * batch_size :]
        else:
            yield 0, true, pred

    for i, t_b, p_b in iterate():
        tensorflow.reset_default_graph()
        with tensorflow.Session() as s:
            sa_graph = losses.masked_spectral_distance(t_b, p_b)
            sa_b = 1 - s.run(sa_graph)
            sa[i * batch_size : i * batch_size + sa_b.shape[0]] = sa_b
    sa = numpy.nan_to_num(sa)
    return sa


def prediction(data, batch_size=600):
    assert "sequence_integer" in data
    assert "intensities_pred" in data
    assert "precursor_charge_onehot" in data

    sequence_lengths = numpy.count_nonzero(data["sequence_integer"], axis=1)
    intensities = data["intensities_pred"]
    charges = list(data["precursor_charge_onehot"].argmax(axis=1) + 1)

    intensities[intensities < 0] = 0
    intensities = normalize_base_peak(intensities)
    intensities = reshape_dims(intensities)
    intensities = mask_outofrange(intensities, sequence_lengths)
    intensities = mask_outofcharge(intensities, charges)
    intensities = reshape_flat(intensities)
    data["intensities_pred"] = intensities

    if "intensities_raw" in data:
        data["spectral_angle"] = get_spectral_angle(
            data["intensities_raw"], data["intensities_pred"], batch_size=batch_size
        )
    return data

In [6]:
# Losses from https://github.com/kusterlab/prosit/blob/master/prosit/losses.py

def masked_spectral_distance(true, pred):
    # Note, fragment ions that cannot exists (i.e. y20 for a 7mer) must have the value  -1.
    import tensorflow
    import keras.backend as k

    epsilon = k.epsilon()
    pred_masked = ((true + 1) * pred) / (true + 1 + epsilon)
    true_masked = ((true + 1) * true) / (true + 1 + epsilon)
    pred_norm = k.l2_normalize(true_masked, axis=-1)
    true_norm = k.l2_normalize(pred_masked, axis=-1)
    product = k.sum(pred_norm * true_norm, axis=1)
    arccos = tensorflow.acos(product)
    return 2 * arccos / numpy.pi


losses = {"masked_spectral_distance": masked_spectral_distance}


def get(loss_name):
    if loss_name in losses:
        return losses[loss_name]
    else:
        return loss_name

In [7]:
# Match from https://github.com/kusterlab/prosit/blob/master/prosit/match.py

def read_attribute(row, attribute):
    if " " not in str(row[attribute]):
        return []
    else:
        return [float(m) for m in row[attribute].split(" ")]


def peptide_parser(p):
    if p[0] == "(":
        raise ValueError("sequence starts with '('")
    n = len(p)
    i = 0
    while i < n:
        if i < n - 3 and p[i + 1] == "(":
            j = p[i + 2 :].index(")")
            offset = i + j + 3
            yield p[i:offset]
            i = offset
        else:
            yield p[i]
            i += 1


def get_forward_backward(peptide):
    amino_acids = peptide_parser(peptide)
    masses = [AMINO_ACID[a] for a in amino_acids]
    forward = np.cumsum(masses)
    backward = np.cumsum(list(reversed(masses)))
    return forward, backward


def get_tolerance(theoretical, mass_analyzer):
    if mass_analyzer in TOLERANCE:
        tolerance, unit = TOLERANCE[mass_analyzer]
        if unit == "ppm":
            return theoretical * float(tolerance) / 10 ** 6
        elif unit == "da":
            return float(tolerance)
        else:
            raise ValueError("unit {} not implemented".format(unit))
    else:
        raise ValueError("no tolerance implemented for {}".format(mass_analyzer))


def is_in_tolerance(theoretical, observed, mass_analyzer):
    mz_tolerance = get_tolerance(theoretical, mass_analyzer)
    lower = observed - mz_tolerance
    upper = observed + mz_tolerance
    return theoretical >= lower and theoretical <= upper


def binarysearch(masses_raw, theoretical, mass_analyzer):
    lo, hi = 0, len(masses_raw) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        if is_in_tolerance(theoretical, masses_raw[mid], mass_analyzer):
            return mid
        elif masses_raw[mid] < theoretical:
            lo = mid + 1
        elif theoretical < masses_raw[mid]:
            hi = mid - 1
    return None


def match(row, ion_types, max_charge=DEFAULT_MAX_CHARGE):
    masses_observed = read_attribute(row, "masses_raw")
    intensities_observed = read_attribute(row, "intensities_raw")
    forward_sum, backward_sum = get_forward_backward(row.modified_sequence[1:-1])
    _max_charge = row.charge if row.charge <= max_charge else max_charge
    matches = []
    for charge_index in range(_max_charge):
        d = {
            "masses_raw": [],
            "masses_theoretical": [],
            "intensities_raw": [],
            "matches": [],
        }
        charge = charge_index + 1
        annotations = get_annotation(
            forward_sum, backward_sum, charge, ion_types
        )
        for annotation, mass_t in annotations.items():
            index = binarysearch(masses_observed, mass_t, row.mass_analyzer)
            if index is not None:
                d["masses_raw"].append(masses_observed[index])
                d["intensities_raw"].append(intensities_observed[index])
                d["masses_theoretical"].append(mass_t)
                d["matches"].append(annotation)
        matches.append(d)
    return matches


def c_lambda(matches, charge, attr):
    def mapping(i):
        charge_index = int(charge - 1)
        m = matches[i]
        if charge_index < len(m):
            try:
                s = ";".join(map(str, m[charge_index][attr]))
            except:
                raise ValueError(m[charge_index][attr])
        else:
            s = ""
        return s

    return mapping


def augment(df, ion_types, charge_max):
    matches = {}
    for i, row in df.iterrows():
        matches[i] = match(row, ion_types, charge_max)

    # augment dataframe and write
    for charge in range(1, charge_max + 1):
        df["matches_charge{}".format(charge)] = df.index.map(
            c_lambda(matches, charge, "matches")
        )
        df["masses_the_charge{}".format(charge)] = df.index.map(
            c_lambda(matches, charge, "masses_theoretical")
        )
        df["masses_raw_charge{}".format(charge)] = df.index.map(
            c_lambda(matches, charge, "masses_raw")
        )
        df["intensities_raw_charge{}".format(charge)] = df.index.map(
            c_lambda(matches, charge, "intensities_raw")
        )

    return df


In [8]:
# Tensorization from https://github.com/kusterlab/prosit/blob/master/prosit/tensorize.py

def stack(queue):
    listed = collections.defaultdict(list)
    for t in queue.values():
        if t is not None:
            for k, d in t.items():
                listed[k].append(d)
    stacked = {}
    for k, d in listed.items():
        if isinstance(d[0], list):
            stacked[k] = [item for sublist in d for item in sublist]
        else:
            stacked[k] = np.vstack(d)
    return stacked


def get_numbers(vals, dtype=float):
    a = np.array(vals).astype(dtype)
    return a.reshape([len(vals), 1])


def get_precursor_charge_onehot(charges):
    array = np.zeros([len(charges), max(CHARGES)], dtype=int)
    for i, precursor_charge in enumerate(charges):
        array[i, precursor_charge - 1] = 1
    return array


def get_sequence_integer(sequences):
    array = np.zeros([len(sequences), MAX_SEQUENCE], dtype=int)
    for i, sequence in enumerate(sequences):
        for j, s in enumerate(peptide_parser(sequence)):
            array[i, j] = ALPHABET[s]
    return array


def parse_ion(string):
    ion_type = ION_TYPES.index(string[0])
    if ("-") in string:
        ion_n, suffix = string[1:].split("-")
    else:
        ion_n = string[1:]
        suffix = ""
    return ion_type, int(ion_n) - 1, NLOSSES.index(suffix)


def get_mz_applied(df, ion_types="yb"):
    ito = {it: ION_OFFSET[it] for it in ion_types}

    def calc_row(row):
        array = np.zeros([MAX_ION, len(ION_TYPES), len(NLOSSES), len(CHARGES)])
        fw, bw = get_forward_backward(row.modified_sequence)
        for z in range(row.precursor_charge):
            zpp = z + 1
            annotation = get_annotation(fw, bw, zpp, ito)
            for ion, mz in annotation.items():
                it, _in, nloss = parse_ion(ion)
                array[_in, it, nloss, z] = mz
        return [array]

    mzs_series = df.apply(calc_row, 1)
    out = np.squeeze(np.stack(mzs_series))
    if len(out.shape) == 4:
        out = out.reshape([1] + list(out.shape))
    return out


def csv(df):
    df.reset_index(drop=True, inplace=True)
    assert "modified_sequence" in df.columns
    assert "collision_energy" in df.columns
    assert "precursor_charge" in df.columns
    data = {
        "collision_energy_aligned_normed": get_numbers(df.collision_energy) / 100.0,
        "sequence_integer": get_sequence_integer(df.modified_sequence),
        "precursor_charge_onehot": get_precursor_charge_onehot(df.precursor_charge),
        "masses_pred": get_mz_applied(df),
    }
    nlosses = 1
    z = 3
    lengths = (data["sequence_integer"] > 0).sum(1)

    masses_pred = get_mz_applied(df)
    masses_pred = cap(masses_pred, nlosses, z)
    masses_pred = mask_outofrange(masses_pred, lengths)
    masses_pred = mask_outofcharge(masses_pred, df.precursor_charge)
    masses_pred = reshape_flat(masses_pred)
    data["masses_pred"] = masses_pred

    return data

In [26]:
test_df = pd.DataFrame({
    "modified_sequence": ["ACDEFGHIK", "LM(ox)NPQRSTV"],
    "collision_energy": [30, 35],
    "precursor_charge": [2, 3],
})

data = csv(test_df)
inputs = dict()
inputs['peptide_sequences'] = data['sequence_integer'].astype(np.float32)
inputs['precursor_charges'] = data['precursor_charge_onehot'].astype(np.float32)
inputs['normalized_collision_energies'] = data['collision_energy_aligned_normed'].astype(np.float32)

In [27]:
outputs = session.run(None, inputs)

In [35]:
outputs[0].shape

(2, 174)